# Comparison of R and Python MIMS Implementations

This notebook compares minutely MIMS units computed using the R MIMSunit package and the Python implementation.

**Analysis includes:**
1. Loading both R and Python MIMS outputs for all participants
2. Per-subject correlation analysis with date × hour heatmaps
3. Aggregated correlation analysis across all subjects
4. Statistical comparison and visualization


In [1]:
# Setup and imports
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100

# Define paths
PROJECT_ROOT = Path('/n/home01/egraff/sample_imputation')
INTERIM_DIR = PROJECT_ROOT / 'data' / 'interim'
PLOTS_DIR = PROJECT_ROOT / 'plots'
PLOTS_DIR.mkdir(exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Interim data directory: {INTERIM_DIR}")
print(f"Plots directory: {PLOTS_DIR}")


Project root: /n/home01/egraff/sample_imputation
Interim data directory: /n/home01/egraff/sample_imputation/data/interim
Plots directory: /n/home01/egraff/sample_imputation/plots


## 1. Load R and Python MIMS Data


In [2]:
def load_python_mims(participant_id, interim_dir):
    """
    Load Python MIMS data for a participant.
    
    Parameters
    ----------
    participant_id : str
        Participant identifier
    interim_dir : Path
        Path to interim data directory
    
    Returns
    -------
    pd.DataFrame or None
        DataFrame with columns 'timestamp' (datetime) and 'mims' (float)
    """
    filepath = interim_dir / f"{participant_id}_python_mims.csv"
    if not filepath.exists():
        print(f"Warning: Python MIMS file not found for {participant_id}: {filepath}")
        return None
    
    df = pd.read_csv(filepath)
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
    return df


def load_r_mims(participant_id, interim_dir):
    """
    Load R MIMS data for a participant.
    
    Parameters
    ----------
    participant_id : str
        Participant identifier
    interim_dir : Path
        Path to interim data directory
    
    Returns
    -------
    pd.DataFrame or None
        DataFrame with columns 'timestamp' (datetime) and 'mims' (float)
    """
    filepath = interim_dir / f"{participant_id}_R_mims.csv"
    if not filepath.exists():
        print(f"Warning: R MIMS file not found for {participant_id}: {filepath}")
        return None
    
    df = pd.read_csv(filepath)
    # Parse timestamp with explicit format for R output
    # Format: "2022-03-21T13:46:00.000000+0000"
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%dT%H:%M:%S.%f%z', utc=True)
    return df


def discover_participants(interim_dir):
    """
    Discover all participants with Python MIMS data.
    
    Parameters
    ----------
    interim_dir : Path
        Path to interim data directory
    
    Returns
    -------
    list of str
        List of participant IDs
    """
    python_mims_files = list(interim_dir.glob("*_python_mims.csv"))
    participants = [f.stem.replace("_python_mims", "") for f in python_mims_files]
    return sorted(participants)


# Discover all participants
participants = discover_participants(INTERIM_DIR)
print(f"\nFound {len(participants)} participants: {', '.join(participants)}")

# Load data for all participants
python_mims_data = {}
r_mims_data = {}

for participant_id in participants:
    print(f"\nLoading data for {participant_id}...")
    
    # Load Python MIMS
    py_df = load_python_mims(participant_id, INTERIM_DIR)
    if py_df is not None:
        python_mims_data[participant_id] = py_df
        print(f"  Python MIMS: {len(py_df):,} epochs")
    
    # Load R MIMS
    r_df = load_r_mims(participant_id, INTERIM_DIR)
    if r_df is not None:
        r_mims_data[participant_id] = r_df
        print(f"  R MIMS: {len(r_df):,} epochs")

print(f"\n{'='*70}")
print(f"Loaded Python MIMS for {len(python_mims_data)} participants")
print(f"Loaded R MIMS for {len(r_mims_data)} participants")
print(f"{'='*70}")



Found 6 participants: 3si9xdvl, bn4j8yj9, gq36edfi, ubrmuu2r, xwyd72l9, zg5uqa5l

Loading data for 3si9xdvl...
  Python MIMS: 9,580 epochs

Loading data for bn4j8yj9...
  Python MIMS: 10,810 epochs

Loading data for gq36edfi...
  Python MIMS: 9,916 epochs

Loading data for ubrmuu2r...
  Python MIMS: 10,902 epochs

Loading data for xwyd72l9...
  Python MIMS: 11,580 epochs

Loading data for zg5uqa5l...
  Python MIMS: 10,152 epochs

Loaded Python MIMS for 6 participants
Loaded R MIMS for 0 participants


In [3]:
# Display summary statistics
summary_data = []

for participant_id in participants:
    if participant_id in python_mims_data and participant_id in r_mims_data:
        py_df = python_mims_data[participant_id]
        r_df = r_mims_data[participant_id]
        
        summary_data.append({
            'Participant': participant_id,
            'Python Epochs': len(py_df),
            'R Epochs': len(r_df),
            'Python Mean MIMS': py_df['mims'].mean(),
            'R Mean MIMS': r_df['mims'].mean(),
            'Python Time Range': f"{py_df['timestamp'].min()} to {py_df['timestamp'].max()}",
            'R Time Range': f"{r_df['timestamp'].min()} to {r_df['timestamp'].max()}"
        })

summary_df = pd.DataFrame(summary_data)
print("\nData Summary:")
print(summary_df.to_string(index=False))



Data Summary:
Empty DataFrame
Columns: []
Index: []


## 2. Per-Subject Correlation Analysis

For each participant, we:
1. Merge R and Python MIMS on timestamp (inner join)
2. Group by date and hour
3. Compute Pearson correlation for each date-hour combination
4. Display as a heatmap with date on Y-axis and hour on X-axis


In [4]:
def compute_per_subject_correlation(participant_id, python_df, r_df):
    """Compute correlation between R and Python MIMS for a participant."""
    # Rename columns to avoid conflicts
    python_df = python_df.rename(columns={'mims': 'mims_python'})
    r_df = r_df.rename(columns={'mims': 'mims_r'})
    
    # Merge on timestamp (inner join to get only matching timestamps)
    merged = python_df.merge(r_df, on='timestamp', how='inner')
    
    if len(merged) == 0:
        print(f"Warning: No matching timestamps for {participant_id}")
        return None
    
    # Extract date and hour
    merged['date'] = merged['timestamp'].dt.date
    merged['hour'] = merged['timestamp'].dt.hour
    
    return merged


def compute_hourly_correlation(merged_df):
    """Compute Pearson correlation for each date-hour combination."""
    def _compute_correlation(group):
        if len(group) < 2:
            return np.nan
        # Check for constant values
        if group['mims_python'].nunique() <= 1 or group['mims_r'].nunique() <= 1:
            return np.nan
        # Compute Pearson correlation
        corr, _ = pearsonr(group['mims_python'], group['mims_r'])
        return corr
    
    # Group by date and hour, compute correlation
    corr_df = merged_df.groupby(['date', 'hour']).apply(_compute_correlation).reset_index()
    corr_df.columns = ['date', 'hour', 'correlation']
    
    # Pivot to create heatmap-ready format
    pivot = corr_df.pivot(index='date', columns='hour', values='correlation')
    
    return pivot


def plot_correlation_heatmap(pivot_df, participant_id, save_path=None):
    """Plot correlation heatmap for a participant."""
    fig_height = max(6, 0.4 * len(pivot_df.index) + 2)
    fig, ax = plt.subplots(figsize=(18, fig_height))
    
    sns.heatmap(
        pivot_df,
        ax=ax,
        cmap='coolwarm',
        vmin=-1,
        vmax=1,
        center=0,
        linewidths=0.3,
        linecolor='white',
        cbar_kws={'label': 'Pearson r'},
        annot=False
    )
    
    ax.set_title(f'{participant_id}: Python vs R MIMS Correlation (Date × Hour)', 
                 fontsize=14, fontweight='bold', pad=15)
    ax.set_xlabel('Hour of Day (UTC)', fontsize=12)
    ax.set_ylabel('Date', fontsize=12)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"  Saved heatmap to {save_path}")
    
    plt.show()


# Process each participant
correlation_results = {}

for participant_id in participants:
    if participant_id not in python_mims_data or participant_id not in r_mims_data:
        print(f"\nSkipping {participant_id}: missing data")
        continue
    
    print(f"\nProcessing {participant_id}...")
    
    # Merge data
    merged = compute_per_subject_correlation(
        participant_id, 
        python_mims_data[participant_id].copy(), 
        r_mims_data[participant_id].copy()
    )
    
    if merged is None:
        continue
    
    print(f"  Matched {len(merged):,} timestamps")
    
    # Compute overall correlation
    overall_corr, overall_pval = pearsonr(merged['mims_python'], merged['mims_r'])
    print(f"  Overall correlation: r={overall_corr:.4f}, p={overall_pval:.4e}")
    
    # Compute hourly correlation
    pivot = compute_hourly_correlation(merged)
    
    # Plot heatmap
    save_path = PLOTS_DIR / f"{participant_id}_r_python_correlation_heatmap.png"
    plot_correlation_heatmap(pivot, participant_id, save_path)
    
    # Store results
    correlation_results[participant_id] = {
        'merged': merged,
        'pivot': pivot,
        'overall_corr': overall_corr,
        'overall_pval': overall_pval
    }

print(f"\n{'='*70}")
print(f"Generated correlation heatmaps for {len(correlation_results)} participants")
print(f"{'='*70}")



Skipping 3si9xdvl: missing data

Skipping bn4j8yj9: missing data

Skipping gq36edfi: missing data

Skipping ubrmuu2r: missing data

Skipping xwyd72l9: missing data

Skipping zg5uqa5l: missing data

Generated correlation heatmaps for 0 participants


In [5]:
# Summary table of per-subject correlations
subject_summary = []

for participant_id, results in correlation_results.items():
    subject_summary.append({
        'Participant': participant_id,
        'N Matched': len(results['merged']),
        'Overall Correlation': results['overall_corr'],
        'P-value': results['overall_pval'],
        'Mean Python MIMS': results['merged']['mims_python'].mean(),
        'Mean R MIMS': results['merged']['mims_r'].mean(),
        'Std Python MIMS': results['merged']['mims_python'].std(),
        'Std R MIMS': results['merged']['mims_r'].std()
    })

subject_summary_df = pd.DataFrame(subject_summary)
print("\nPer-Subject Correlation Summary:")
print(subject_summary_df.to_string(index=False))

# Display statistics
print(f"\nMean correlation across subjects: {subject_summary_df['Overall Correlation'].mean():.4f}")
print(f"Median correlation across subjects: {subject_summary_df['Overall Correlation'].median():.4f}")
print(f"Min correlation: {subject_summary_df['Overall Correlation'].min():.4f}")
print(f"Max correlation: {subject_summary_df['Overall Correlation'].max():.4f}")



Per-Subject Correlation Summary:
Empty DataFrame
Columns: []
Index: []


KeyError: 'Overall Correlation'

## 3. Aggregated Correlation Analysis

Stack all observations across all participants and compute overall Pearson correlation.


In [ ]:
# Stack all matched observations
all_observations = []

for participant_id, results in correlation_results.items():
    merged = results['merged'].copy()
    merged['participant_id'] = participant_id
    all_observations.append(merged)

# Combine all observations
stacked_df = pd.concat(all_observations, ignore_index=True)

print(f"Total stacked observations: {len(stacked_df):,}")
print(f"Participants: {stacked_df['participant_id'].nunique()}")

# Remove any rows with missing values
stacked_df = stacked_df.dropna(subset=['mims_python', 'mims_r'])
print(f"After removing NaN: {len(stacked_df):,} observations")

# Compute overall correlation
overall_corr, overall_pval = pearsonr(stacked_df['mims_python'], stacked_df['mims_r'])

print(f"\n{'='*70}")
print(f"Overall Correlation (all subjects combined):")
print(f"  Pearson r: {overall_corr:.6f}")
print(f"  P-value: {overall_pval:.4e}")
print(f"  N observations: {len(stacked_df):,}")
print(f"{'='*70}")


In [ ]:
# Scatter plot of R vs Python MIMS (all observations)
fig, ax = plt.subplots(figsize=(10, 10))

# Sample if too many points
if len(stacked_df) > 10000:
    plot_df = stacked_df.sample(n=10000, random_state=42)
    title_suffix = f" (sample of 10,000 from {len(stacked_df):,})"
else:
    plot_df = stacked_df
    title_suffix = f" (N={len(stacked_df):,})"

ax.scatter(plot_df['mims_r'], plot_df['mims_python'], 
           alpha=0.3, s=10, c='steelblue', edgecolors='none')

# Add diagonal reference line
max_val = max(plot_df['mims_r'].max(), plot_df['mims_python'].max())
ax.plot([0, max_val], [0, max_val], 'r--', linewidth=2, alpha=0.7, label='y=x')

ax.set_xlabel('R MIMS', fontsize=12)
ax.set_ylabel('Python MIMS', fontsize=12)
ax.set_title(f'R vs Python MIMS{title_suffix}\nr={overall_corr:.4f}, p={overall_pval:.2e}', 
             fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
save_path = PLOTS_DIR / 'r_python_mims_scatter_all.png'
plt.savefig(save_path, dpi=150, bbox_inches='tight')
print(f"Saved scatter plot to {save_path}")
plt.show()


In [ ]:
# Create correlation bar plot showing agreement per participant
participant_correlations = subject_summary_df.set_index('Participant')['Overall Correlation']

fig, ax = plt.subplots(figsize=(12, 6))

participant_correlations.plot(kind='bar', ax=ax, color='steelblue', edgecolor='black')

ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax.axhline(y=1, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Perfect correlation')
ax.set_xlabel('Participant ID', fontsize=12)
ax.set_ylabel('Pearson Correlation (r)', fontsize=12)
ax.set_title('R vs Python MIMS Correlation by Participant', fontsize=14, fontweight='bold')
ax.set_ylim(-1, 1.05)
ax.grid(True, alpha=0.3, axis='y')
ax.legend()

plt.xticks(rotation=45, ha='right')
plt.tight_layout()

save_path = PLOTS_DIR / 'r_python_correlation_by_participant.png'
plt.savefig(save_path, dpi=150, bbox_inches='tight')
print(f"Saved correlation bar plot to {save_path}")
plt.show()


In [ ]:
# Analyze differences between R and Python MIMS
stacked_df['difference'] = stacked_df['mims_r'] - stacked_df['mims_python']
stacked_df['abs_difference'] = np.abs(stacked_df['difference'])
stacked_df['percent_difference'] = 100 * stacked_df['difference'] / (stacked_df['mims_python'] + 1e-10)

print(f"\n{'='*70}")
print("Difference Analysis (R - Python):")
print(f"{'='*70}")
print(f"Mean difference: {stacked_df['difference'].mean():.6f}")
print(f"Median difference: {stacked_df['difference'].median():.6f}")
print(f"Std of difference: {stacked_df['difference'].std():.6f}")
print(f"Mean absolute difference: {stacked_df['abs_difference'].mean():.6f}")
print(f"Median absolute difference: {stacked_df['abs_difference'].median():.6f}")
print(f"\nPercentile of differences:")
print(f"  5th percentile: {stacked_df['difference'].quantile(0.05):.6f}")
print(f"  25th percentile: {stacked_df['difference'].quantile(0.25):.6f}")
print(f"  50th percentile: {stacked_df['difference'].quantile(0.50):.6f}")
print(f"  75th percentile: {stacked_df['difference'].quantile(0.75):.6f}")
print(f"  95th percentile: {stacked_df['difference'].quantile(0.95):.6f}")


In [ ]:
# Bland-Altman plot
stacked_df['mean_mims'] = (stacked_df['mims_r'] + stacked_df['mims_python']) / 2

fig, ax = plt.subplots(figsize=(12, 8))

# Sample if too many points
if len(stacked_df) > 10000:
    plot_df = stacked_df.sample(n=10000, random_state=42)
else:
    plot_df = stacked_df

ax.scatter(plot_df['mean_mims'], plot_df['difference'], 
           alpha=0.3, s=10, c='steelblue', edgecolors='none')

# Add reference lines
mean_diff = stacked_df['difference'].mean()
std_diff = stacked_df['difference'].std()

ax.axhline(y=mean_diff, color='red', linestyle='-', linewidth=2, label=f'Mean: {mean_diff:.4f}')
ax.axhline(y=mean_diff + 1.96*std_diff, color='red', linestyle='--', linewidth=1.5, 
           label=f'+1.96 SD: {mean_diff + 1.96*std_diff:.4f}')
ax.axhline(y=mean_diff - 1.96*std_diff, color='red', linestyle='--', linewidth=1.5, 
           label=f'-1.96 SD: {mean_diff - 1.96*std_diff:.4f}')
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8, alpha=0.5)

ax.set_xlabel('Mean of R and Python MIMS', fontsize=12)
ax.set_ylabel('Difference (R - Python)', fontsize=12)
ax.set_title('Bland-Altman Plot: R vs Python MIMS', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
save_path = PLOTS_DIR / 'r_python_bland_altman.png'
plt.savefig(save_path, dpi=150, bbox_inches='tight')
print(f"Saved Bland-Altman plot to {save_path}")
plt.show()


## 4. Summary and Conclusions


In [ ]:
print("\n" + "="*70)
print("SUMMARY: R vs Python MIMS Comparison")
print("="*70)

print(f"\nDatasets Analyzed:")
print(f"  Number of participants: {len(correlation_results)}")
print(f"  Total matched observations: {len(stacked_df):,}")

print(f"\nOverall Agreement:")
print(f"  Pearson correlation: r = {overall_corr:.6f}")
print(f"  P-value: {overall_pval:.4e}")

print(f"\nPer-Subject Correlations:")
print(f"  Mean: {subject_summary_df['Overall Correlation'].mean():.4f}")
print(f"  Median: {subject_summary_df['Overall Correlation'].median():.4f}")
print(f"  Range: [{subject_summary_df['Overall Correlation'].min():.4f}, {subject_summary_df['Overall Correlation'].max():.4f}]")

print(f"\nDifference Statistics (R - Python):")
print(f"  Mean difference: {stacked_df['difference'].mean():.6f}")
print(f"  Median absolute difference: {stacked_df['abs_difference'].median():.6f}")
print(f"  95% of differences within: [{stacked_df['difference'].quantile(0.025):.6f}, {stacked_df['difference'].quantile(0.975):.6f}]")

print(f"\nGenerated Figures:")
print(f"  Per-subject heatmaps: {len(correlation_results)} files")
print(f"  Scatter plot: r_python_mims_scatter_all.png")
print(f"  Correlation by participant: r_python_correlation_by_participant.png")
print(f"  Bland-Altman plot: r_python_bland_altman.png")

print("\n" + "="*70)
print("Analysis complete!")
print("="*70)


In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

import sys
SRC_PATH = Path('/n/home01/egraff/sample_imputation/src')
if str(SRC_PATH) not in sys.path:
    sys.path.append(str(SRC_PATH))

# Configuration
NOTEBOOK_DIR = Path('.').resolve()
INTERIM_DIR = (NOTEBOOK_DIR / '../data/interim').resolve()
DATA_ROOT = Path('/n/home01/egraff/sample_imputation/data/raw')

print(f'Interim directory: {INTERIM_DIR}')
print(f'Data root: {DATA_ROOT}')



In [ ]:
# Get list of participants
SUBJECT_IDS = sorted(
    d.name for d in DATA_ROOT.iterdir()
    if d.is_dir() and d.name != '__MACOSX'
)

if not SUBJECT_IDS:
    raise FileNotFoundError(f'No participant directories found in {DATA_ROOT}')

print(f'Found {len(SUBJECT_IDS)} participants: {", ".join(SUBJECT_IDS)}')



In [ ]:
# Load Python MIMS files
PYTHON_MIMS_SUFFIX = 'python_mims'
python_mims: dict[str, pd.DataFrame] = {}

for subject_id in SUBJECT_IDS:
    python_path = INTERIM_DIR / f'{subject_id}_{PYTHON_MIMS_SUFFIX}.csv'
    if python_path.exists():
        df = pd.read_csv(python_path)
        df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
        python_mims[subject_id] = df
        print(f'Loaded Python MIMS for {subject_id}: {len(df):,} epochs')
    else:
        print(f'Warning: Python MIMS file not found for {subject_id}: {python_path}')

print(f'\nLoaded Python MIMS for {len(python_mims)} participants')



In [ ]:
# Load R MIMS files
R_MIMS_SUFFIX = 'R_mims'
r_mims: dict[str, pd.DataFrame] = {}

for subject_id in SUBJECT_IDS:
    r_path = INTERIM_DIR / f'{subject_id}_{R_MIMS_SUFFIX}.csv'
    if r_path.exists():
        df = pd.read_csv(r_path)
        df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
        r_mims[subject_id] = df
        print(f'Loaded R MIMS for {subject_id}: {len(df):,} epochs')
    else:
        print(f'Warning: R MIMS file not found for {subject_id}: {r_path}')

print(f'\nLoaded R MIMS for {len(r_mims)} participants')



In [ ]:
# Check which participants have both R and Python MIMS
common_subjects = set(python_mims.keys()) & set(r_mims.keys())
print(f'Participants with both R and Python MIMS: {len(common_subjects)}')
print(f'Subjects: {", ".join(sorted(common_subjects))}')

if len(common_subjects) == 0:
    raise ValueError('No participants have both R and Python MIMS data')



In [ ]:
def compute_hourly_correlation(df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute correlation for each date/hour combination.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with columns: timestamp, mims_python, mims_r
        
    Returns:
    --------
    pd.DataFrame
        Pivot table with date as index, hour as columns, correlation as values
    """
    df = df.copy()
    df['date'] = df['timestamp'].dt.date
    df['hour'] = df['timestamp'].dt.hour
    
    def _corr(group: pd.DataFrame) -> float:
        if len(group) < 2:
            return np.nan
        if group['mims_python'].nunique() <= 1 or group['mims_r'].nunique() <= 1:
            return np.nan
        return group['mims_python'].corr(group['mims_r'])
    
    corr_df = (
        df.groupby(['date', 'hour'], observed=True)
        .apply(_corr)
        .reset_index(name='correlation')
    )
    
    pivot = corr_df.pivot(index='date', columns='hour', values='correlation')
    return pivot



In [ ]:
# Per-subject correlation analysis and heatmaps
correlation_pivots: dict[str, pd.DataFrame] = {}

for subject_id in sorted(common_subjects):
    print(f'\nProcessing {subject_id}...')
    
    # Merge R and Python MIMS on timestamp
    py_df = python_mims[subject_id].rename(columns={'mims': 'mims_python'})
    r_df = r_mims[subject_id].rename(columns={'mims': 'mims_r'})
    
    merged = (
        py_df[['timestamp', 'mims_python']]
        .merge(r_df[['timestamp', 'mims_r']], on='timestamp', how='inner')
        .sort_values('timestamp')
    )
    
    if len(merged) == 0:
        print(f'  No overlapping timestamps for {subject_id}')
        continue
    
    print(f'  Merged {len(merged):,} overlapping timestamps')
    
    # Compute overall correlation
    overall_corr, p_value = pearsonr(merged['mims_python'], merged['mims_r'])
    print(f'  Overall correlation: {overall_corr:.4f} (p={p_value:.2e})')
    
    # Compute hourly correlation heatmap
    pivot = compute_hourly_correlation(merged)
    correlation_pivots[subject_id] = pivot
    
    # Create heatmap
    fig_height = max(3, 0.4 * len(pivot.index) + 2)
    fig, ax = plt.subplots(figsize=(18, fig_height))
    
    sns.heatmap(
        pivot,
        ax=ax,
        cmap='coolwarm',
        vmin=-1,
        vmax=1,
        center=0,
        linewidths=0.3,
        linecolor='white',
        cbar_kws={'label': 'Pearson r'},
        annot=False,
        fmt='.2f'
    )
    
    ax.set_title(f'{subject_id} - Python vs R MIMS Correlation (Date × Hour)', fontsize=14, pad=20)
    ax.set_xlabel('Hour of Day (UTC)', fontsize=12)
    ax.set_ylabel('Date', fontsize=12)
    
    plt.tight_layout()
    plt.show()



In [ ]:
# Stack all observations across all participants
all_observations = []

for subject_id in sorted(common_subjects):
    py_df = python_mims[subject_id].rename(columns={'mims': 'mims_python'})
    r_df = r_mims[subject_id].rename(columns={'mims': 'mims_r'})
    
    merged = (
        py_df[['timestamp', 'mims_python']]
        .merge(r_df[['timestamp', 'mims_r']], on='timestamp', how='inner')
    )
    
    merged['participant_id'] = subject_id
    all_observations.append(merged)

# Combine all observations
stacked_df = pd.concat(all_observations, ignore_index=True)

# Remove rows with missing values
stacked_df = stacked_df.dropna(subset=['mims_python', 'mims_r'])

print(f'Total observations across all participants: {len(stacked_df):,}')
print(f'Participants: {stacked_df["participant_id"].nunique()}')

# Compute overall Pearson correlation
overall_corr, p_value = pearsonr(stacked_df['mims_python'], stacked_df['mims_r'])
print(f'\nOverall Pearson correlation: {overall_corr:.4f}')
print(f'P-value: {p_value:.2e}')



In [ ]:
# Create correlation matrix heatmap for all participants
# Compute correlation for each participant
participant_corrs = {}

for subject_id in sorted(common_subjects):
    py_df = python_mims[subject_id].rename(columns={'mims': 'mims_python'})
    r_df = r_mims[subject_id].rename(columns={'mims': 'mims_r'})
    
    merged = (
        py_df[['timestamp', 'mims_python']]
        .merge(r_df[['timestamp', 'mims_r']], on='timestamp', how='inner')
        .dropna(subset=['mims_python', 'mims_r'])
    )
    
    if len(merged) >= 2:
        corr, _ = pearsonr(merged['mims_python'], merged['mims_r'])
        participant_corrs[subject_id] = corr

# Create correlation matrix (each participant's R-Python correlation)
corr_matrix = pd.DataFrame(
    [[participant_corrs.get(subj, np.nan)] for subj in sorted(participant_corrs.keys())],
    index=sorted(participant_corrs.keys()),
    columns=['R-Python Correlation']
)

# Create heatmap
fig, ax = plt.subplots(figsize=(8, max(4, 0.5 * len(corr_matrix) + 2)))

sns.heatmap(
    corr_matrix,
    ax=ax,
    cmap='coolwarm',
    vmin=-1,
    vmax=1,
    center=0,
    annot=True,
    fmt='.3f',
    cbar_kws={'label': 'Pearson r'},
    linewidths=0.5,
    linecolor='white'
)

ax.set_title('R vs Python MIMS Correlation by Participant', fontsize=14, pad=20)
ax.set_xlabel('')
ax.set_ylabel('Participant ID', fontsize=12)

plt.tight_layout()
plt.show()

print(f'\nParticipant correlations:')
for subj, corr in sorted(participant_corrs.items()):
    print(f'  {subj}: {corr:.4f}')



In [ ]:
# Create scatter plot of R vs Python MIMS for all participants
fig, ax = plt.subplots(figsize=(10, 10))

# Plot each participant with different color
colors = plt.cm.tab10(np.linspace(0, 1, len(common_subjects)))

for idx, subject_id in enumerate(sorted(common_subjects)):
    py_df = python_mims[subject_id].rename(columns={'mims': 'mims_python'})
    r_df = r_mims[subject_id].rename(columns={'mims': 'mims_r'})
    
    merged = (
        py_df[['timestamp', 'mims_python']]
        .merge(r_df[['timestamp', 'mims_r']], on='timestamp', how='inner')
        .dropna(subset=['mims_python', 'mims_r'])
    )
    
    if len(merged) > 0:
        ax.scatter(
            merged['mims_python'],
            merged['mims_r'],
            alpha=0.3,
            s=1,
            label=subject_id,
            color=colors[idx]
        )

# Add diagonal line
max_val = max(
    stacked_df['mims_python'].max(),
    stacked_df['mims_r'].max()
)
ax.plot([0, max_val], [0, max_val], 'r--', alpha=0.5, label='y=x')

ax.set_xlabel('Python MIMS', fontsize=12)
ax.set_ylabel('R MIMS', fontsize=12)
ax.set_title('R vs Python MIMS Comparison (All Participants)', fontsize=14)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

